# Connect to SVV Using psycopg2

In [1]:
import psycopg2

def get_svv_connection():
    con = psycopg2.connect(dbname = 'svv', host ='svv-rs-prod-bi.cjddijbnvfpr.eu-west-1.redshift.amazonaws.com', \
                         port = 5439, user = 'ryanw', password = 'hT6Y3TeZZUFdvLj')
    
    return con

# Import Pandas and Time

In [2]:
import pandas as pd

In [34]:
from time import time

# Query the sandbox.hubp_viewing_summary_es Table

In [3]:
query_data = "Select top 100 * from sandbox.hubp_viewing_summary_es"

df_viewing_summary = pd.DataFrame()

try: 
    con = get_svv_connection()

    cur = con.cursor()
    cur.execute(query_data)

    colnames = [desc[0] for desc in cur.description]
    df_viewing_summary = pd.DataFrame(cur.fetchall(), columns=colnames)
    
except Exception as inst:
    print(inst)
finally:
    cur.close()
    con.close()

# Check the Resulting Data Frame

In [4]:
df_viewing_summary.head()

,viewer_id,title,created,gender,age,barb,lifestage,days_since_last_view,target_hubplus,isp_list,...,number_of_weeks_monthly,total_visits_monthly,total_progs_monthly,short_subgenre_monthly_sg,total_streaming_monthly_sg,average_streaming_monthly_sg,medium_streaming_monthly_sg,number_of_days_monthly_sg,number_of_weeks_monthly_sg,total_visits_monthly_sg
0,00001235-b046-4591-96fe-172654977f54,None,2018-08-13 08:55:33.893,M,80.0,ad.axa.me,Active,71,0,Virgin Media,...,1,1,1,08-2018_CRIME AND THRILLER,39.6430000000,39.6430000000,39.64300000000,1,1,1
1,00005939-09b6-4550-9b42-5441089ffd15,Mr,2016-06-05 14:18:21.340,M,26.0,a3.ad.m3.me,Active,63,0,Virgin Media,...,1,1,1,09-2018_FACTUAL REALITY,2577.9520000000,2577.9520000000,2577.95200000000,1,1,1
2,000071d1-7452-4d64-a1c6-8c710f6a6755,Mrs,2018-06-17 18:51:44.544,F,30.0,a3.ad.axa.h5.hc.hw.w3.wo,Active,6,0,BT,...,"2,3,1,1","2,5,1,1","1,3,1,1","06-2018_LONG-RUNNING DRAMA,08-2018_CELEBRITY F...","5460.2280000000,24.0000000000,15113.1800000000...","1820.0760000000,24.0000000000,3022.6360000000,...","1339.48900000000,24.00000000000,2770.000000000...","2,1,4,1,1,1","2,1,2,1,1,1","2,1,4,1,1,1"
3,00007b23-89ce-451b-a034-1905c79f4f51,Miss,2018-07-14 17:16:04.943,F,27.0,a3.ad.axa.h5.hc.hw.w3.wo,Active,74,0,BT;Telefonica O2 UK,...,"2,1","3,1","1,1","07-2018_DATING SHOWS,08-2018_DATING SHOWS","4194.7970000000,1032.0710000000","1398.2656666666,1032.0710000000","1320.93600000000,1032.07100000000","2,1","2,1","3,1"
4,00008c05-80f7-4292-b6e0-2a11a4277483,Mrs,2013-01-02 00:00:00.000,F,NaN,ad.wo,Inactive,110,0,Virgin Media,...,1,1,1,07-2018_DATING SHOWS,3005.8590000000,3005.8590000000,3005.85900000000,1,1,1


# Restrict the Data Frame to the two necessary Columns

In [15]:
df_viewing_summary = df_viewing_summary.loc[:,['month_year_total','total_streaming_monthly']]
df_viewing_summary.head()

,month_year_total,total_streaming_monthly
0,08-2018,39.6430000000
1,09-2018,2577.9520000000
2,"06-2018,08-2018,09-2018,10-2018","5460.2280000000,16476.1800000000,2749.26700000..."
3,"07-2018,08-2018","4194.7970000000,1032.0710000000"
4,07-2018,3005.8590000000


# Convert the Data Frame to a Dictionary

In [30]:
def maximumConsumptionMonth(months, streaming):
    
    keys = months.split(',')
    values = [float(x) for x in streaming.split(',')]
    
    #splits the commma separated values on each line and converts the values from a string to a float
    
    vals_dict = dict(zip(keys,values))
    
    #combines key and values into a dictionary
    
    return Counter(vals_dict).most_common(1)[0][0]

    #returns the month on each row that corresponds to the highest consumption


#dict_viewing_summary = df_viewing_summary.set_index('month_year_total').transpose().to_dict(orient='list')
#dict_viewing_summary

# Create a new Column called max_month to store the month with the maximum consumption

In [35]:
start = time()
df_viewing_summary['max_month'] = list(map(maximumConsumptionMonth, df_viewing_summary['month_year_total'],
                                           df_viewing_summary['total_streaming_monthly']))
print(time()-start)

0.0039980411529541016


# Check that the Function has worked properly

In [33]:
df_viewing_summary[['max_month', 'month_year_total', 'total_streaming_monthly']]

,max_month,month_year_total,total_streaming_monthly
0,08-2018,08-2018,39.6430000000
1,09-2018,09-2018,2577.9520000000
2,08-2018,"06-2018,08-2018,09-2018,10-2018","5460.2280000000,16476.1800000000,2749.26700000..."
3,07-2018,"07-2018,08-2018","4194.7970000000,1032.0710000000"
4,07-2018,07-2018,3005.8590000000
5,10-2018,"08-2018,09-2018,10-2018","11616.1970000000,11339.5980000000,22509.091000..."
6,07-2018,"01-2018,04-2018,05-2018,07-2018,08-2018,09-201...","603.0320000000,3797.2110000000,6858.9940000000..."
7,06-2018,"06-2018,07-2018","28960.2470000000,24999.2580000000"
8,07-2018,07-2018,59531.1450000000
9,07-2018,"07-2018,08-2018","38226.4510000000,17802.9070000000"
